In [1]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 36.3 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully u

In [2]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

In [4]:
df = pd.read_csv("procData.csv")
df = df.sample(frac=1).reset_index(drop=True) # shuffling since train-test-val does not
df.head()

,title,files,description,category,fileAmount,more100Files,fileSize
0,[EMBER] Made in Abyss (2022) (Season 2) [BDRip...,S02E01-The Compass Pointed to the Darkness [89...,\n ## `Made in Abyss S02 10...,Anime-English-translated,12,0,23.136221
1,[EMBER] Isekai Meikyuu de Harem wo S01E05 (Unc...,[EMBER] Isekai Meikyuu de Harem wo - 05 [Uncen...,\n ## `Isekai Meikyuu de Ha...,Anime-English-translated,1,0,20.478205
2,[GenmCorp] Ultraman Taiga Tri-Squad Voice Dram...,[GenmCorp] Ultraman Taiga Tri-Squad Voice Dram...,\n #### No description.\n ...,Anime-English-translated,1,0,17.891860
3,Platinum End - S01E20 - MULTi 1080p WEB x264 -...,Platinum End - S01E20 - MULTi 1080p WEB x264 -...,\n [**Platinum End - S01**]...,Anime-English-translated,1,0,21.199881
4,[mal lu zen] Kinsou no Vermeil - 08 [720p] 10-...,[mal lu zen] Kinsou no Vermeil - 08 [Uncensore...,\n | Anime | Kinsou no...,Anime-English-translated,1,0,19.343583


In [5]:
# df = df[(df.category != 'Pictures')] # small categories
# df = df[(df.category != 'Software')]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70294 entries, 0 to 70293
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         70294 non-null  object 
 1   files         70294 non-null  object 
 2   description   70294 non-null  object 
 3   category      70294 non-null  object 
 4   fileAmount    70294 non-null  int64  
 5   more100Files  70294 non-null  int64  
 6   fileSize      70294 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 3.8+ MB


In [6]:
train, val, test = np.split(df.sample(frac=1), [int(0.8 * len(df)), int(0.9 * len(df))])

In [7]:
print(len(train), len(val), len(test), len(df))

56235 7029 7030 70294


In [8]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df['category']), y=df['category']))

In [9]:
weights = {}
for index, w in enumerate(class_weights):
  weights[index] = w
weights 

{0: 59.926683716965044,
 1: 0.16583545265382965,
 2: 0.2113870035514845,
 3: 0.8522137626691237,
 4: 2.197099456148028,
 5: 3.701827373742693,
 6: 0.9706434686550677,
 7: 67.78592092574735,
 8: 0.6030248179190008,
 9: 3.9989759927181705,
 10: 22.9718954248366,
 11: 16.152113970588236,
 12: 0.8404351984696318,
 13: 44.461733080328905,
 14: 295.3529411764706,
 15: 153.1459694989107,
 16: 23.361249584579596}

In [10]:
np.unique(df['category'])

array(['Anime-Anime Music Video', 'Anime-English-translated',
       'Anime-Non-English-translated', 'Anime-Raw', 'Audio-Lossless',
       'Audio-Lossy', 'Literature-English-translated',
       'Literature-Non-English-translated', 'Literature-Raw',
       'Live Action-English-translated',
       'Live Action-Idol/Promotional Video',
       'Live Action-Non-English-translated', 'Live Action-Raw',
       'Pictures-Graphics', 'Pictures-Photos', 'Software-Applications',
       'Software-Games'], dtype=object)

In [11]:
def df_to_dataset(dataframe, shuffle=True, batch_size=64):
  df = dataframe.copy()
  labels = df.pop('category')
  df = {key: value[:,tf.newaxis] for key, value in df.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [12]:
train_data = df_to_dataset(train)
test_data = df_to_dataset(test)
validation_data = df_to_dataset(val)

<ipython-input-11-95dc31fad2a0>:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in df.items()}


In [13]:
train['category'].value_counts()

Anime-English-translated              20052
Anime-Non-English-translated          15575
Literature-Raw                         5475
Live Action-Raw                        3953
Anime-Raw                              3890
Literature-English-translated          3399
Audio-Lossless                         1491
Audio-Lossy                             882
Live Action-English-translated          823
Live Action-Non-English-translated      203
Software-Games                          147
Live Action-Idol/Promotional Video      141
Pictures-Graphics                        77
Anime-Anime Music Video                  54
Literature-Non-English-translated        45
Software-Applications                    17
Pictures-Photos                          11
Name: category, dtype: int64

In [14]:
catVals = np.unique(df['category'])
table = tf.lookup.StaticHashTable(
    initializer = tf.lookup.KeyValueTensorInitializer(
        keys = tf.constant(catVals),
        values = list(range(len(catVals)))
    ),
    default_value = -1,
    name = "target_encoding"
)

In [15]:
catVals

array(['Anime-Anime Music Video', 'Anime-English-translated',
       'Anime-Non-English-translated', 'Anime-Raw', 'Audio-Lossless',
       'Audio-Lossy', 'Literature-English-translated',
       'Literature-Non-English-translated', 'Literature-Raw',
       'Live Action-English-translated',
       'Live Action-Idol/Promotional Video',
       'Live Action-Non-English-translated', 'Live Action-Raw',
       'Pictures-Graphics', 'Pictures-Photos', 'Software-Applications',
       'Software-Games'], dtype=object)

In [16]:
@tf.function
def target(x):
  return table.lookup(x)

In [17]:
def fetch(features, labels):
  return features, tf.one_hot(target(labels), len(catVals))

In [18]:
train_data_f = train_data.map(fetch)
test_data_f = test_data.map(fetch)
validation_data_f = validation_data.map(fetch)

In [19]:
embedding = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
hub_layer = hub.KerasLayer(embedding, output_shape = 512, input_shape = [], dtype=tf.string, trainable=True)

In [20]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  if dtype == 'string':
    index = tf.keras.layers.StringLookup(max_tokens=max_tokens)
  else:
    index = tf.keras.layers.IntegerLookup(max_tokens=max_tokens)

  feature_ds = dataset.map(lambda x, y: x[name])
  index.adapt(feature_ds)
  encoder = tf.keras.layers.CategoryEncoding(num_tokens=index.vocabulary_size())
  return lambda feature: encoder(index(feature))

In [21]:
def get_normalization_layer(name, dataset):
  normalizer = tf.keras.layers.Normalization(axis=None)
  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)
  return normalizer

In [22]:
all_inputs = []
encoded_features = []

In [23]:
for header in ['fileSize']: # fileAmount
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_data_f)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# for header in ['more100Files']:
#   num_cat_col = tf.keras.Input(shape=(1,), name=header)
#   categ_layer = get_category_encoding_layer(header, train_data_f, "int64")
#   encoded_categ_col = categ_layer(num_cat_col)
#   all_inputs.append(num_cat_col)
#  encoded_features.append(encoded_categ_col)

In [24]:
for header in ['title', 'description', 'files']:
  text_col = tf.keras.Input(shape=(), name=header, dtype='string')
  encoded_text_col = hub_layer(text_col)
  all_inputs.append(text_col)
  encoded_features.append(encoded_text_col)

In [25]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = all_features
for i in range(3):
  x = tf.keras.layers.Dense(16, activation='relu')(x)
  # x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(len(catVals), activation='softmax')(x)

In [26]:
model = tf.keras.Model(all_inputs, output)

In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(), # deleted from_logits=True
              metrics=["categorical_accuracy",
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
                        tfa.metrics.F1Score(num_classes=len(catVals),
                                            average='macro',
                                            threshold=0.5)])

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fileSize (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 title (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 description (InputLayer)       [(None,)]            0           []                               
                                                                                                  
 files (InputLayer)             [(None,)]            0           []                               
                                                                                              

In [30]:
history = model.fit(train_data_f, epochs=5, validation_data=validation_data_f, class_weight=weights)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['fileAmount', 'more100Files'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


879/879 [==============================] - 624s 595ms/step - loss: 2.4421 - categorical_accuracy: 0.4299 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.0000e+00 - val_loss: 1.7884 - val_categorical_accuracy: 0.5183 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 2/5
879/879 [==============================] - 260s 296ms/step - loss: 1.7262 - categorical_accuracy: 0.5799 - precision: 0.9977 - recall: 0.2320 - f1_score: 0.1259 - val_loss: 1.0775 - val_categorical_accuracy: 0.6273 - val_precision: 0.9967 - val_recall: 0.4352 - val_f1_score: 0.2062
Epoch 3/5
879/879 [==============================] - 248s 282ms/step - loss: 1.1443 - categorical_accuracy: 0.7435 - precision: 0.9933 - recall: 0.5198 - f1_score: 0.3728 - val_loss: 0.8114 - val_categorical_accuracy: 0.8469 - val_precision: 0.9911 - val_recall: 0.5547 - val_f1_score: 0.5086
Epoch 4/5
879/879 [==============================] - 245s 279ms/step - loss: 0.7236 - categorical_accuracy:

In [31]:
model.evaluate(test_data_f)

110/110 [==============================] - 11s 100ms/step - loss: 0.4292 - categorical_accuracy: 0.9189 - precision: 0.9721 - recall: 0.8529 - f1_score: 0.7099


[0.4292038679122925,
 0.9189189076423645,
 0.9721141457557678,
 0.8529160618782043,
 0.7099101543426514]

In [32]:
model.evaluate(validation_data_f)

110/110 [==============================] - 12s 107ms/step - loss: 0.4158 - categorical_accuracy: 0.9222 - precision: 0.9753 - recall: 0.8594 - f1_score: 0.6984


[0.41578561067581177,
 0.9221795201301575,
 0.9752987027168274,
 0.859439492225647,
 0.6983510851860046]

In [41]:
model.save("trainedModel_final_yessubcat")

In [33]:
# test_x = test_data_f.unbatch().map(lambda x, y: x)
test_x = test_data_f.unbatch().map(lambda x, y: x)
test_y = test_data_f.unbatch().map(lambda x, y: y)

In [35]:
test_predicted = model.predict(test_x)

7030/7030 [==============================] - 214s 31ms/step


In [36]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [37]:
test_y = np.array([x for x in test_y])

In [38]:
print(classification_report(test_y.argmax(1), test_predicted.argmax(1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.40      0.38      0.39      2457
           2       0.36      0.32      0.34      1998
           3       0.11      0.12      0.12       453
           4       0.08      0.08      0.08       194
           5       0.12      0.12      0.12       119
           6       0.12      0.12      0.12       427
           7       0.00      0.00      0.00         8
           8       0.19      0.19      0.19       720
           9       0.05      0.07      0.06        98
          10       0.02      0.05      0.03        21
          11       0.00      0.05      0.01        19
          12       0.15      0.15      0.15       480
          13       0.00      0.00      0.00         7
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         7
          16       0.05      0.08      0.06        13

    accuracy              

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
print(tf.math.confusion_matrix(test_y.argmax(1), test_predicted.argmax(1)))

tf.Tensor(
[[  0   5   1   0   0   0   0   0   0   0   0   0   1   0   0   0   0]
 [  0 942 557 154  60  38 138   3 226  58  18  78 168   4  10   0   3]
 [  0 635 632 109  47  33 114   3 176  33  15  60 128   7   4   0   2]
 [  0 150  95  53  14   4  25   0  45   8   5  16  31   2   4   0   1]
 [  0  63  45  13  15   2   9   0  25   2   0   4  10   1   0   0   5]
 [  0  38  23   8   4  14   8   0  11   2   0   3   8   0   0   0   0]
 [  0 144  91  29   9   5  53   1  34   6   4  18  28   2   1   0   2]
 [  0   4   0   0   0   0   1   0   1   0   0   0   2   0   0   0   0]
 [  0 199 163  56  18  10  42   1 134  11   4  26  48   2   3   0   3]
 [  0  31  27   6   1   3   2   0   9   7   0   4   8   0   0   0   0]
 [  0   9   3   1   1   1   2   0   2   0   1   0   1   0   0   0   0]
 [  0   5   3   5   1   0   1   0   2   0   0   1   1   0   0   0   0]
 [  0 132 105  31  22   6  30   1  39  10   6  22  74   0   0   0   2]
 [  0   2   4   1   0   0   0   0   0   0   0   0   0   0   0   0 

In [44]:
!zip -r "/content/trainedModel_final_allcats.zip" "/content/trainedModel_final_yessubcat"

  adding: content/trainedModel_final_yessubcat/ (stored 0%)
  adding: content/trainedModel_final_yessubcat/assets/ (stored 0%)
  adding: content/trainedModel_final_yessubcat/fingerprint.pb (stored 0%)
  adding: content/trainedModel_final_yessubcat/variables/ (stored 0%)
  adding: content/trainedModel_final_yessubcat/variables/variables.index (deflated 80%)
  adding: content/trainedModel_final_yessubcat/variables/variables.data-00000-of-00001 (deflated 40%)
  adding: content/trainedModel_final_yessubcat/saved_model.pb (deflated 70%)
  adding: content/trainedModel_final_yessubcat/keras_metadata.pb (deflated 90%)


In [45]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
!mv "/content/trainedModel_final_allcats.zip" "/content/drive/My Drive/trainedModel_final_allcats.zip"